In [1]:
%pip install anomalib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 257 kB 17.5 MB/s 
     |████████████████████████████████| 41 kB 360 kB/s 
     |████████████████████████████████| 585 kB 62.7 MB/s 
     |████████████████████████████████| 431 kB 47.2 MB/s 
     |████████████████████████████████| 1.9 MB 57.6 MB/s 
     |████████████████████████████████| 551 kB 61.4 MB/s 
     |████████████████████████████████| 13.8 MB 55.7 MB/s 
     |████████████████████████████████| 1.8 MB 60.2 MB/s 
     |████████████████████████████████| 446 kB 79.1 MB/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
     |████████████████████████████████| 169 kB 10.2 MB/s 
     |████████████████████████████████| 9.4 MB 24.9 MB/s 
     |████████████████████████████████| 79 kB 2.5 MB/s 
     |████████████████████████████████| 419 kB 19.4 MB/s 
     |████████████████████████████████| 182 kB 68.4 MB/s 
     |████████████████████████████

In [2]:


!git clone https://github.com/openvinotoolkit/anomalib

Cloning into 'anomalib'...
remote: Enumerating objects: 24980, done.
remote: Counting objects: 100% (1087/1087), done.
remote: Compressing objects: 100% (517/517), done.
remote: Total 24980 (delta 688), reused 877 (delta 562), pack-reused 23893
Receiving objects: 100% (24980/24980), 54.90 MiB | 25.79 MiB/s, done.
Resolving deltas: 100% (15150/15150), done.


In [3]:
from pathlib import Path
from typing import Any, Dict

import numpy as np
from IPython.display import display
from PIL import Image
from pytorch_lightning import Trainer
from torchvision.transforms import ToPILImage

from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.models import get_model
from anomalib.pre_processing.transforms import Denormalize
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
MODEL = "ganomaly"  # 'padim', 'cflow', 'stfpm', 'ganomaly', 'dfkde', 'patchcore'
CONFIG_PATH = f"/content/anomalib/anomalib/models/{MODEL}/config.yaml"
with open(file=CONFIG_PATH, mode="r", encoding="utf-8") as file:
    print(file.read())


dataset:
  name: training
  format: folder
  path: /content/drive/MyDrive/test
  normal_dir: /content/drive/MyDrive/test/NEtrain/NE
  abnormal_dir: /content/drive/MyDrive/test/normal/AE
  normal_test_dir: null # name of the folder containing normal test images.
  task: classification # classification or segmentation
  mask: null
  extensions: null
  split_ratio: 0.2 # ratio of the normal images that will be used to create a test split
  image_size: 256
  train_batch_size: 32
  test_batch_size: 32
  num_workers: 8
  transform_config:
    train: null
    val: null
  create_validation_set: true
  tiling:
    apply: false
    tile_size: null
    stride: null
    remove_border_count: 0
    use_random_tiling: False
    random_tile_count: 16

 

model:
  name: ganomaly
  latent_vec_size: 100
  n_features: 64
  extra_layers: 0
  add_final_conv: true
  early_stopping:
    patience: 3
    metric: image_AUROC
    mode: max
  lr: 0.0002
  beta1: 0.5
  beta2: 0.999
  wadv: 1
  wcon: 50
  wenc: 1
  

In [6]:
# pass the config file to model, callbacks and datamodule
config = get_configurable_parameters(config_path=CONFIG_PATH)
config["dataset"]["path"] = "/content/drive/MyDrive"  # or wherever the MVTec dataset is stored.

In [7]:
datamodule = get_datamodule(config)
# datamodule.setup()  # Downloads the dataset if it's not in the specified `root` directory
datamodule.prepare_data()  # Create train/val/test/prediction sets.



In [8]:
model = get_model(config)
callbacks = get_callbacks(config)

/usr/local/lib/python3.8/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.8/dist-packages/anomalib/utils/callbacks/__init__.py:143: UserWarning: Export option: None not found. Defaulting to no model export
  warnings.warn(f"Export option: {config.optimization.export_mode} not found. Defaulting to no model export")


In [9]:
# start training
trainer = Trainer(**config.trainer,callbacks=callbacks)
trainer.fit(model=model, datamodule=datamodule)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/epoch/training_epoch_loop.py:52: LightningDeprecationWarning: Setting `max_steps = None` is deprecated in v1.5 and will no longer be supported in v1.7. Use `max_steps = -1` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
INFO:pytorch_

Training: 0it [00:00, ?it/s]

In [ ]:
# load best model from checkpoint before evaluating
load_model_callback = LoadModelCallback(weights_path=trainer.checkpoint_callback.best_model_path)
trainer.callbacks.insert(0, load_model_callback)
trainer.test(model=model, datamodule=datamodule)